In [1]:
import ee
import geemap



In [2]:
import geeft
from geersd import Sentinel2


In [3]:
ee.Initialize()

In [4]:
aoi = ee.FeatureCollection('users/ryangilberthamilton/ParksColab/PNP/pnp_aoi').geometry().bounds()

In [5]:
def add_ndvi(image: ee.image.Image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI').float())

def ndvi_from_expression(image: ee.Image):
    return image.addBands(
        image.expression(
            expression="(NIR - Red) / (NIR + Red)",
            map_={"NIR": image.select('B8'), "Red": image.select('B4')}
        ).rename('NDVI').float()
    )

doy = lambda x: x.set({'doy': ee.Image(x).date().getRelative('day', 'year').add(1).int()})

In [6]:
s2_sr = (
    Sentinel2
    .surface_reflectance()
    .filterBounds(aoi)
    .filterDate("2018", '2021')
    .filterCloud(1)
    .applyCloudMask()
    .map(add_ndvi)
)


s2_sr.first()


In [7]:
s2_sr.aggregate_array('system:time_start').distinct()

In [8]:
ft = geeft.compute(s2_sr, 'NDVI', 4)

In [9]:
ft.bandNames()

In [18]:
sr_vis = {
    'min': -1,
    'max': 1,
    'bands': ['NDVI']
}

ft_vis = {
    'bands': ['phase_1', 'amp_1', 'amp_2']
}

In [19]:
Map = geemap.Map()

Map.addLayer(s2_sr, {'min': -1, 'max': 1, 'bands': ['NDVI']}, 'True Colour')
Map.addLayer(ft, ft_vis, "Fourier Transform")
Map.centerObject(aoi)
Map.addLayerControl()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [20]:
task = ee.batch.Export.image.toDrive(
        image=ft,
        description="pcft-export",
        folder="ft-export",
        fileNamePrefix="pcft-export-",
        fileDimensions=[2048, 2048],
        scale=10,
        crs="EPSG:4326",
        region=aoi,
        skipEmptyTiles=True
    )

task.start()
print(f"Task: {task.id}")

Task: TLHM3JCXPOTXALOGOSY4KIHI
